In [2]:
%pip install lightgbm

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 9.7 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
train_df = pd.read_csv('/home/codespace/.jupyter/NeurIPS/Data/train.csv')
ts = pd.read_csv('/home/codespace/.jupyter/NeurIPS/Data/test.csv')
ss = pd.read_csv('/home/codespace/.jupyter/NeurIPS/Data/sample_submission.csv')

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split


# 1. split off 20% for dev_test
temp_df, dev_test = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42,  # for reproducibility
    shuffle=True
)

# 2. split the remaining 80% into 75% train / 25% valid → 0.6 / 0.2 overall
dev_train, dev_val = train_test_split(
    temp_df,
    test_size=0.25,  # 0.25 * 0.8 = 0.2 of the original
    random_state=42,
    shuffle=True
)

# Verify sizes
print(f"Total rows:   {len(train_df)}")
print(f"Dev train:    {len(dev_train)} ({len(dev_train)/len(train_df):.2%})")
print(f"Dev valid:    {len(dev_val)} ({len(dev_val)/len(train_df):.2%})")
print(f"Dev test:     {len(dev_test)} ({len(dev_test)/len(train_df):.2%})")
print(f"Polymer example:{dev_train['SMILES'].to_list()[:3]}")
print(f"Columns:{dev_train.columns}")

Total rows:   7973
Dev train:    4783 (59.99%)
Dev valid:    1595 (20.01%)
Dev test:     1595 (20.01%)
Polymer example:['*Nc1ccc(CC(CC(C)(C)c2ccc(N*)cc2)=C(C)C)cc1', '*CC(*)(CC(=O)OC)C(=O)OC12CC3CC(C)(CC(C)(C3)C1)C2', '*OP(=O)(Oc1c(Cl)cc(Cl)cc1Cl)Oc1c(Cl)c(Cl)c(*)c(Cl)c1Cl']
Columns:Index(['id', 'SMILES', 'Tg', 'FFV', 'Tc', 'Density', 'Rg'], dtype='object')


In [5]:

char_dic = {
    '<pad>': 0,
    '#': 1,   # Triple bond
    '%': 2,   # Two-digit ring closure (e.g., '%10')
    '(': 3,   # Branch opening
    ')': 4,   # Branch closing
    '*': 5,   # Wildcard atom (used in BigSMILES for polymer repeating units)
    '+': 6,   # Positive charge
    '-': 7,   # Negative charge
    '0': 8,   # Ring closure digit
    '1': 9,
    '2': 10,
    '3': 11,
    '4': 12,
    '5': 13,
    '6': 14,
    '7': 15,
    '8': 16,
    '9': 17,
    '.': 18,  # Disconnected structures
    '/': 19,  # Stereochemistry (up bond)
    '1': 20,  # Stereochemistry (down bond)
    'A': 21
}

In [6]:
%pip install torch_molecule tqdm

from tqdm.notebook import tqdm as notebook_tqdm
import tqdm
tqdm.tqdm = notebook_tqdm
tqdm.trange = notebook_tqdm

from torch_molecule import LSTMMolecularPredictor
from torch_molecule.utils.search import ParameterType, ParameterSpec

search_parameters = {
    "output_dim": ParameterSpec(ParameterType.INTEGER, (8, 32)),
    "LSTMunits": ParameterSpec(ParameterType.INTEGER, (30, 120)),
    "learning_rate": ParameterSpec(ParameterType.LOG_FLOAT, (1e-4, 1e-2)),
}

lstm = LSTMMolecularPredictor(
    task_type="regression",
    num_task=5,
    batch_size=192,
    epochs=200,
    verbose=True
)

print("Model initialized successfully")
X_train = dev_train['SMILES'].to_list()
y_train = dev_train[['Tg', 'FFV', 'Tc', 'Density', 'Rg']].to_numpy()
X_val = dev_val['SMILES'].to_list()
y_val = dev_val[['Tg', 'FFV', 'Tc', 'Density', 'Rg']].to_numpy()
lstm.autofit(
    X_train = X_train,
    y_train = y_train,
    X_val = X_val,
    y_val = y_val,
    search_parameters=search_parameters,
    n_trials = 10 # number of times searching the best hyper-parameters
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 32.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  4/19 [Mako]dict]  WARNING: The script mako-render is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-09-12 23:32:25,608] A new study created in memory with name: LSTMMolecularPredictor_optimization


Model initialized successfully

Parameter Search Configuration:
--------------------------------------------------

 Parameters being searched:
  • LSTMunits: [30, 120]
  • learning_rate: [0.0001, 0.01]
  • output_dim: [8, 32]

 Fixed parameters (not being searched):
  • batch_size: 192
  • device: cpu
  • epochs: 200
  • evaluate_criterion: <function mean_absolute_error at 0x7c4277d6d800>
  • evaluate_higher_better: False
  • evaluate_name: mae
  • fitting_epoch: 0
  • fitting_loss: []
  • input_dim: 54
  • loss_criterion: MSELoss()
  • max_input_len: 200
  • num_task: 5
  • patience: 50
  • scheduler_factor: 0.5
  • scheduler_patience: 5
  • task_type: regression
  • use_lr_scheduler: False
  • verbose: True
  • weight_decay: 0.0

--------------------------------------------------

Starting hyperparameter optimization using mae metric
Direction: minimize
Number of trials: 10


  0%|          | 0/10 [00:00<?, ?it/s]--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/optuna/progress_bar.py", line 24, in emit
    tqdm.write(msg)
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/std.py", line 720, in write
    with cls.external_write_mode(file=file, nolock=nolock):
  File "/usr/local/python/3.12.1/lib/python3.12/contextlib.py", line 144, in __exit__
    next(self.gen)
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/std.py", line 750, in external_write_mode
    inst.refresh(nolock=True)
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/std.py", line 1347, in refresh
    self.display()
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/notebook.py", line 156, in display
    ltext, pbar, rtext = self.container.children
                         ^^^^^^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'container'
Call s

[W 2025-09-12 23:32:25,619] Trial 0 failed with parameters: {'output_dim': 24, 'LSTMunits': 62, 'learning_rate': 0.00040784205413030126} because of the following error: ImportError('IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html').
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/torch_molecule/predictor/lstm/modeling_lstm.py", line 304, in objective
    self.fit(X_train, y_train, X_val, y_val)
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/torch_molecule/predictor/lstm/modeling_lstm.py", line 423, in fit
    train_dataset = self._convert_to_pytorch_data(X_train, y_train)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/optuna/progress_bar.py", line 24, in emit
    tqdm.write(msg)
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/std.py", line 720, in write
    with cls.external_write_mode(file=file, nolock=nolock):
  File "/usr/local/python/3.12.1/lib/python3.12/contextlib.py", line 144, in __exit__
    next(self.gen)
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/std.py", line 750, in external_write_mode
    inst.refresh(nolock=True)
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/std.py", line 1347, in refresh
    self.display()
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/notebook.py", line 156, in display
    ltext, pbar, rtext = self.container.children
                         ^^^^^^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'container'
Call stack:
  File "<frozen runpy>", line 19

[W 2025-09-12 23:32:25,760] Trial 8 failed with parameters: {'output_dim': 12, 'LSTMunits': 81, 'learning_rate': 0.008546086735469081} because of the following error: ImportError('IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html').
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/torch_molecule/predictor/lstm/modeling_lstm.py", line 304, in objective
    self.fit(X_train, y_train, X_val, y_val)
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/torch_molecule/predictor/lstm/modeling_lstm.py", line 423, in fit
    train_dataset = self._convert_to_pytorch_data(X_train, y_train)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12.1

LSTMMolecularPredictor(device=device(type='cpu'), model_name='LSTMMolecularPredictor', model_class=<class 'torch_molecule.predictor.lstm.model.LSTM'>, model=LSTM(
  (embedding): Embedding(54, 23)
  (lstm1): LSTM(23, 93, batch_first=True, bidirectional=True)
  (lstm2): LSTM(186, 93, batch_first=True, bidirectional=True)
  (timedist_dense): Linear(in_features=186, out_features=46, bias=True)
  (relu): ReLU()
  (fc): Linear(in_features=9200, out_features=5, bias=True)
), is_fitted_=False, num_task=5, task_type='regression', input_dim=54, output_dim=23, LSTMunits=93, max_input_len=200, batch_size=192, epochs=200, loss_criterion=MSELoss(), evaluate_criterion=<function mean_absolute_error at 0x7c4277d6d800>, evaluate_higher_better=False, learning_rate=0.0014392372138228079, weight_decay=0.0, patience=50, use_lr_scheduler=False, scheduler_factor=0.5, scheduler_patience=5, verbose=True, fitting_loss=[], fitting_epoch=0)

In [13]:
# Load sample submission
sample_sub = ss
print(sample_sub.head())

# Load test set
print(ts.head())

# Prepare test SMILES list
X_test = ts['SMILES'].to_list()

# Predict using the trained LSTM model
lstm_preds = lstm.predict(X_test)['prediction']
preds = lstm_preds

# Build the submission DataFrame
submission_df = sample_sub.copy()
submission_df[['Tg', 'FFV', 'Tc', 'Density', 'Rg']] = preds

print('submission_df', submission_df)

           id  Tg  FFV  Tc  Density  Rg
0  1109053969   0    0   0        0   0
1  1422188626   0    0   0        0   0
2  2032016830   0    0   0        0   0
           id                                             SMILES
0  1109053969  *Oc1ccc(C=NN=Cc2ccc(Oc3ccc(C(c4ccc(*)cc4)(C(F)...
1  1422188626  *Oc1ccc(C(C)(C)c2ccc(Oc3ccc(C(=O)c4cccc(C(=O)c...
2  2032016830  *c1cccc(OCCCCCCCCOc2cccc(N3C(=O)c4ccc(-c5cccc6...


AttributeError: This model is not fitted yet. Call 'fit' before using it.

In [ ]:
# save to CSV
submission_df.to_csv('submission.csv', index=False)